In [16]:
# -*- coding: UTF-8 -*-

#########################IMPORTS############################################
import re
import sys
import string
#########################CONSTANTS##########################################

In [17]:
debug=0
debug_now=1

In [18]:
xml_glossary_header = """
<?xml version="1.0" encoding="UTF-8"?>
<GLOSSARY>
  <INFO>
		<NAME>%s</NAME>
		<INTRO>%s</INTRO>
		<STUDENTCANPOST>0</STUDENTCANPOST>
		<ALLOWDUPLICATEDENTRIES>0</ALLOWDUPLICATEDENTRIES>
		<DISPLAYFORMAT>dictionary</DISPLAYFORMAT>
		<SHOWSPECIAL>1</SHOWSPECIAL>
		<SHOWALPHABET>1</SHOWALPHABET>
		<SHOWALL>1</SHOWALL>
		<ALLOWCOMMENTS>0</ALLOWCOMMENTS>
		<USEDYNALINK>1</USEDYNALINK>
		<DEFAULTAPPROVAL>1</DEFAULTAPPROVAL>
		<GLOBALGLOSSARY>0</GLOBALGLOSSARY>
		<ENTBYPAGE>25</ENTBYPAGE>
	
		<ENTRIES>
"""

xml_glossary_trailer = """
		</ENTRIES>
	</INFO>
</GLOSSARY>
"""


glossary_entry = """
	<ENTRY>
		<CONCEPT>
			%s
		</CONCEPT>
		<DEFINITION>
			%s
		</DEFINITION>
		<FORMAT>1</FORMAT>
		<USEDYNALINK>0</USEDYNALINK>
		<CASESENSITIVE>0</CASESENSITIVE>
		<FULLMATCH>0</FULLMATCH>
		<TEACHERENTRY>1</TEACHERENTRY>
	</ENTRY>
"""


xml_cloze_header = """
<?xml version="1.0" encoding="UTF-8"?>
<quiz>
	<question type="category">
		<category><text>%s</text></category>
	</question>
"""

xml_cloze_trailer = """
</quiz>
"""

#<!-- question: %d  -->
cloze_question = """
<question type="cloze">
	<name><text>%s</text></name>
	<questiontext>
		<text><![CDATA[Complétez la définition suivante :<br><br>%s<br><br>]]></text>
	</questiontext>
	<generalfeedback><text>%s</text></generalfeedback>
	<shuffleanswers>%d</shuffleanswers>
</question>
"""

general_feedback="Bien reçu."
shuffle_answers=1

In [19]:
#########################VARIABLES##########################################

#########################FUNCTIONS##########################################
def stderrprint(chaine,fin="\n"):
        sys.stderr.write(chaine+fin)

def mots_ligne(line):
        return re.findall(r"[\wâàéèêëîïôùûüçÂÀÉÈÊËÎÏÔÙÛÜÇæœÆŒ'’=~-]+|[.…,;!?:—–()\[\]\/# ""«»<>]", line)

def keys_ligne(line):
    keys=[]
    embedded_keys = re.findall(r"[\[#][\wâàéèêëîïôùûüçÂÀÉÈÊËÎÏÔÙÛÜÇæœÆŒ'’]+[\]#]", line)
    for key in embedded_keys:
        keys.append(key[1:-1])
    return keys

def print_cloze(chaine,fin="\n"):
    if sortie_cloze!="":
        s_cloze.write(chaine+fin)
    else:
        stderrprint(chaine,fin)

def parse_keys(ligne):
    if sac:
        mots=mots_ligne(ligne)
        mots_sac=[]
        for mot in mots:
            if mot in sac:
                mots_sac.append(bracket_mot(mot))
            else:
                    mots_sac.append(mot)
        definition="".join(mots_sac)
    else:
        definition=ligne
    keys=keys_ligne(definition)
    words=mots_ligne(definition)
    cloze_lst=[]
    gloss_lst=[]
    for word in words:
        if not word in ["[","]","#"]:
            gloss_lst.append(word)
            if word in keys:
                cloze_lst.append("{1:SAC:="+word+"}")
            else:
                cloze_lst.append(word)
    gloss_def=normalize_ligne("".join(gloss_lst))
    cloze_def=normalize_ligne("".join(cloze_lst))
    return (gloss_def,cloze_def)

def bracket_mot(mot):
        return "["+mot+"]"
    
def parse_ligne(line):
    champs=line.split(':')
    entry=normalize_ligne(champs[0])
    definition=":".join(champs[1:])
    return [entry,definition]

def add_ligne(entry,line):
    definition=entry[1]+"<br>"+line
    return [entry[0],definition]
    
def store_entree(entry):
    (item,definition)=entry
    (gloss_def,cloze_def)=parse_keys(definition)
    gloss_def=gloss_def.replace("<br>","<![CDATA[<br>]]>")
    if sortie_glossaire!="":
        s_glossary.write(glossary_entry % (item, gloss_def))
    else:
        print glossary_entry % (item, gloss_def)
    if sortie_cloze!="":
        s_cloze.write(cloze_question % ("définition de "+item,item+": "+cloze_def,general_feedback,shuffle_answers))
        
        
def normalize_ligne(line):
    chaine=" ".join(line.strip().split())
    #chaine=chaine.decode('utf-8').replace(u'\xa0',u' ')
    #chaine=chaine.encode('utf-8').replace(' }','}')
    chaine=chaine.replace(' }','}')
    chaine=chaine.replace('( ','(')
    chaine=chaine.replace(' )',')')
    chaine=chaine.replace(' ,',',')
    chaine=chaine.replace(' .','.')
    chaine=chaine.replace("’","'")
    chaine=chaine.replace("' ","'")
    chaine=chaine.replace("   "," ")
    chaine=chaine.replace("  "," ")
    return chaine
    
    
def ligne_vide(line):
    ligne=normalize_ligne(line)
    if len(ligne)==0 or ligne==" ":
        return True
    else:
        return False

In [20]:
entree_glossaire="/Users/gilles/ownCloud/Cours/Sdlcours/glossaire26.gloss"
sortie_glossaire=""
sortie_cloze=""



try:
	glossary_file = open(entree_glossaire)
except IOError:
	print 'I could not open the glossary file', entree_glossaire
	sys.exit()

fichier=glossary_file.readlines()
glossary_file.close()

if sortie_glossaire!="":
	try:
		s_glossary=open(sortie_glossaire,'w')
	except IOError:
		print 'I could not open the glossary writing file', sortie_glossaire
		sys.exit()

if sortie_cloze!="":
	try:
		s_cloze=open(sortie_cloze,'w')
	except IOError:
		print 'I could not open the glossary writing file', sortie_glossaire
		sys.exit()

nom_glossaire="glossaire"
description_glossaire="glossaire"
question_category="Catégorie par défaut"

sac=[]
entree=[]
entete_xml_ecrit=False
entete_glossary_ecrit=False

In [22]:
for line in fichier:
    tampon=line.strip()
    if not tampon.startswith("#"):
        if ligne_vide(tampon) and entree:
            if not entete_xml_ecrit:
                if sortie_glossaire!="":
                    s_glossary.write(xml_glossary_header % (nom_glossaire,description_glossaire))
                else:
                    print xml_glossary_header % (nom_glossaire,description_glossaire)
                if sortie_cloze!="":
                    s_cloze.write(xml_cloze_header % question_category)
                entete_xml_ecrit=True
            store_entree(entree)
            entree=[]
        elif ligne_vide(tampon):
            entree=[]
        else:
            if entree:
                entree=add_ligne(entree,tampon)
            else:
                entree=parse_ligne(tampon)
if sortie_glossaire:
    s_glossary.write(xml_glossary_trailer)
else:
    print xml_glossary_trailer
if sortie_cloze:
    s_cloze.write(xml_cloze_trailer)


<?xml version="1.0" encoding="UTF-8"?>
<GLOSSARY>
  <INFO>
		<NAME>glossaire</NAME>
		<INTRO>glossaire</INTRO>
		<STUDENTCANPOST>0</STUDENTCANPOST>
		<ALLOWDUPLICATEDENTRIES>0</ALLOWDUPLICATEDENTRIES>
		<DISPLAYFORMAT>dictionary</DISPLAYFORMAT>
		<SHOWSPECIAL>1</SHOWSPECIAL>
		<SHOWALPHABET>1</SHOWALPHABET>
		<SHOWALL>1</SHOWALL>
		<ALLOWCOMMENTS>0</ALLOWCOMMENTS>
		<USEDYNALINK>1</USEDYNALINK>
		<DEFAULTAPPROVAL>1</DEFAULTAPPROVAL>
		<GLOBALGLOSSARY>0</GLOBALGLOSSARY>
		<ENTBYPAGE>25</ENTBYPAGE>
	
		<ENTRIES>


	<ENTRY>
		<CONCEPT>
			Mot
		</CONCEPT>
		<DEFINITION>
			Ce terme recouvre plusieurs concepts qui demandent à être distingués :<br />rn<ol>rn <li>la notion de lexème</li>rn <li>la notion de forme fléchie</li>rn <li>la notion d''unité phonologique autonome<br /></li>rn</ol>
		</DEFINITION>
		<FORMAT>1</FORMAT>
		<USEDYNALINK>0</USEDYNALINK>
		<CASESENSITIVE>0</CASESENSITIVE>
		<FULLMATCH>0</FULLMATCH>
		<TEACHERENTRY>1</TEACHERENTRY>
	</ENTRY>


	<ENTRY>
		<CONCEPT>
			Lexème